In [4]:
from langchain_core.messages import HumanMessage
from langchain_core.prompts import PromptTemplate

from langchain.chat_models import QianfanChatEndpoint

prompt = PromptTemplate.from_template("介绍一下{who}。")
content = prompt.format(who='苏轼')
chat = QianfanChatEndpoint()
chat([HumanMessage(content=content)]).content

'苏轼（1037年1月8日—1101年8月24日），字子瞻，号东坡居士，世称苏东坡、苏仙、坡仙，是宋代著名的文学家、书法家、画家和政治家，也是一位美食家和医药爱好者。\n\n苏轼是北宋中期文坛的领袖人物，在诗、词、散文方面取得很高的成就。他的文章纵横恣肆，为“唐宋八大家”之一。他的书法与蔡襄、黄庭坚、米芾合称“宋四家”，影响很大。在绘画方面，他善画山水，自成一家，开创了湖州画派。他的作品有诗文集《东坡七集》和《东坡易传》等。\n\n除此之外，苏轼在医药方面也很有造诣。他是一个受禅宗影响极深的文人，参禅吃素对他养生有好处。他认为药补不如食补，食物是最好的医药。他喜好烹饪，对食品的加工颇有研究，还著有《苏学士食单》和《苏沈良方》等美食专著。\n\n苏轼在任职期间，曾多次被贬或遭流放。但他始终乐观向上，善于从自然万物中寻找乐趣和启迪。他喜爱游览名山大川，常常借物咏怀，抒发情感。他在逆境中不断寻找生活中的美好事物，并用自己的方式将其表现出来，成为了中国古代文学史上的一个重要人物。\n\n总的来说，苏轼是一位才华横溢的文化名人，他对诗词、书法、绘画、医药等多个领域都有深厚的造诣和贡献。'

In [1]:

from libs.community.langchain_community.embeddings.ernie import ErnieEmbeddings
from libs.community.langchain_community.llms.baidu_qianfan_endpoint import QianfanLLMEndpoint
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
import os


vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at coffee shop", "bears like to eat honey"],
    embedding=ErnieEmbeddings(),
    # embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = QianfanLLMEndpoint()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

# resp = chain.invoke("where did harrison work?")
# print(resp)
for chunk in chain.stream("where did harrison work?"):
    print(chunk, end="", flush=True)


KeyboardInterrupt



In [17]:
from operator import itemgetter
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()


template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context":  itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language2"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "where did harrison work", "language2": "chinese"})

'好的，根据提供的信息，以下是中文的回答：\n\nHarrison在Kensho工作。'

In [36]:
from libs.community.langchain_community.chat_models.openai import ChatOpenAI
from langchain.schema import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.runnables import RunnableParallel
from langchain.prompts.prompt import PromptTemplate

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)


_inputs = RunnableParallel(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: get_buffer_string(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
)
_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents,
    "question": lambda x: x["standalone_question"],
}

input_raw = {
        "question": "Really??",
        "chat_history": [
            HumanMessage(content="where did harrison work?"),
            AIMessage(content='Harrison worked at Kensho.'),
        ],
    }

invoke = _inputs.invoke(input_raw)
context__invoke = RunnableParallel(_context).invoke(invoke)
prompt_invoke = ANSWER_PROMPT.invoke(context__invoke)
ChatOpenAI().invoke(prompt_invoke)
# (_context | ANSWER_PROMPT | ChatOpenAI()).invoke(invoke)


AIMessage(content='Yes, it is true that Harrison worked at Kensho.')

In [55]:
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables.base import RunnableLambda
from operator import itemgetter
from libs.langchain.langchain.memory import ConversationBufferMemory

# 首先需要加载缓冲的聊天记录
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)

# 把问题，和聊天记录，整成一个问题
standalone_question = RunnableParallel({
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
})

# 根据整好的问题，从知识库当中检索答案
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}

# 把知识库内容，问题，丢给LLM，然后LLM返回答案，知识库内容保持不变
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(),
    "docs": itemgetter("docs"),
}

input_raw = {"question": "where did harrison work?"}
memory_invoke = loaded_memory.invoke(input_raw)
question_invoke = standalone_question.invoke(memory_invoke)
documents__invoke = RunnableParallel(retrieved_documents).invoke(question_invoke)
result = RunnableParallel(answer).invoke(documents__invoke)
result

memory.save_context(input_raw, {"answer": result["answer"].content})
memory.load_memory_variables({})

{'history': [HumanMessage(content='where did harrison work?'),
  AIMessage(content='Harrison was employed at Kensho.')]}

In [56]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='where did harrison work?'),
  AIMessage(content='Harrison was employed at Kensho.')]}

In [60]:
input_raw2 = {"question": "but where did he really work?"}
final_chain = loaded_memory | standalone_question | retrieved_documents | answer
result = final_chain.invoke(input_raw2)
memory.save_context(input_raw2, {"answer": result["answer"].content})
memory.load_memory_variables({})

{'history': [HumanMessage(content='where did harrison work?'),
  AIMessage(content='Harrison was employed at Kensho.'),
  HumanMessage(content='but where did he really work?'),
  AIMessage(content='Harrison actually worked at Kensho.')]}

In [61]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"person": "毛泽东", "language": "chinese"})

'毛泽东，又被称为主席毛，来自中国湖南省韶山市。'

In [67]:
from langchain_core.runnables import RunnablePassthrough

prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)

model_parser = model | StrOutputParser()

color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser
question_generator = (
    color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)
question_generator.invoke("warm")

ChatPromptValue(messages=[HumanMessage(content='What is the color of Marigold and the flag of Antigua and Barbuda?')])

In [70]:
planner = (
    ChatPromptTemplate.from_template("生成一场正反方的辩论。主题是: {input}")
    | ChatOpenAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "列出正方观点，基于： {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)
arguments_against = (
    ChatPromptTemplate.from_template(
        "列出反方观点，基于： {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "正反:\n{results_1}\n\n反方:\n{results_2}"),
            ("system", "给出一个最终判定，谁赢了"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

planner_invoke = planner.invoke({"input": "品德高于情商"})
RunnableParallel(
    {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
).invoke(planner_invoke)


{'results_1': '正方观点总结：品德高于情商，因为品德决定了一个人如何对待他人和自己，具有高品德的人会采取正确的行为和决策，建立可靠和稳固的人际关系。品德也是社会责任感的表现，高品德的人会参与公益活动并帮助他人。\n\n反方观点总结：情商高于品德，因为情商可以帮助人们更好地处理情感和人际关系，应对压力和挑战，并促进个人的成功。在现代社会中，人际关系和合作能力对个人的成功至关重要，情商高的人更容易与他人合作并建立良好的合作关系。',
 'results_2': '4. 情商可以帮助人们更好地适应现代社会的快节奏和高压力环境。在现代社会中，人们常常面临各种压力和挑战，情商高的人可以更好地应对这些挑战，保持积极的心态和情绪稳定。\n\n5. 情商可以提高个人的领导能力。一个情商高的人通常具备良好的沟通和影响力，能够有效地领导和管理团队，实现集体目标。\n\n6. 情商可以促进个人的自我认知和成长。通过提高情商，人们可以更好地了解自己的情感和行为模式，从而更好地发展个人潜力和实现自我成长。\n\n7. 情商可以帮助人们更好地处理人际关系和冲突。情商高的人通常能够更好地理解他人的情感和需求，并能够通过有效的沟通和解决冲突的能力来维护良好的人际关系。\n\n8. 情商可以提高个人的创造力和创新能力。情商高的人通常能够更好地理解自己和他人的需求，并能够提出创新的解决方案和新的思维方式。\n\n总结：\n\n反方认为情商高于品德，因为情商可以帮助人们更好地适应现代社会的快节奏和高压力环境，提升个人的领导能力，促进自我认知和成长，处理人际关系和冲突，以及提高创造力和创新能力。这些观点强调了情商在个人发展和成功方面的重要性。',
 'original_response': '正方：品德高于情商\n\n1. 品德是一个人的核心价值观和道德准则，它决定了一个人如何对待他人和自己。一个有高尚品德的人会采取正确的行为和决策，从而建立起可靠和稳固的人际关系。\n\n2. 品德可以使一个人更加善良和宽容。一个具有高品德的人在面对困难和挫折时，会以一种积极的态度去应对，而不是通过欺骗或背叛他人来达到自己的目标。\n\n3. 品德是一个人的社会责任感的表现。高品德的人会主动参与公益活动，为社会做出贡献。他们愿意帮助那些需要帮助的人，并且具备良好的团队合作精神。\n\n反方：情商高于品德\n